In [3]:
import boto3
import sagemaker
from sagemaker import get_execution_role
region = boto3.session.Session().region_name

role = get_execution_role()

## Training Job
We're gonna use a training job to load an already trained model and save it as an a AWS compatible model

We won't perform any type of training.

In [8]:
%%writefile train.py
import os

os.system("pip install -U sagemaker")

import numpy as np
import tensorflow as tf
import pandas as pd
import argparse
import tarfile
from tensorflow.keras import backend as K

from sagemaker.session import Session

import boto3

boto_session = boto3.session.Session(region_name=os.environ["REGION"])
sagemaker_session = Session(boto_session=boto_session)
s3 = boto3.client("s3")

# modified from AWS Sagemaker examples
def parse_args():
    
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    
    # data directories
    parser.add_argument('--bucket', type=str),
    parser.add_argument("--prefix", type=str),
    # path to the model inside the zipped file
    parser.add_argument("--zipped_model_path", type=str),
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    
    # model directory: we will use the default set by SageMaker, /opt/ml/model
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    
    return parser.parse_known_args()


if __name__ == "__main__":
        
    args, _ = parse_args()
    bucket = args.bucket
    prefix = args.prefix
    zipped_model_path = args.zipped_model_path
    
    # Download model zipped
    s3.download_file(
        bucket, 
        prefix, 
        "model.gz"
    )
    
    tar = tarfile.open("model.gz")
    tar.extractall()
    tar.close()
    
    # If you have any custom function that your model was trained on
    # you should re-defined it here and pass it to the load model function
    
    def fb_score(y_true, y_pred, beta=2):
        """Performs fb score"""
        y_pred = K.round(y_pred)
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)

        precision = tp / (tp + fp + K.epsilon())
        recall = tp = (tp + fn + K.epsilon())

        fb = (1 + beta) * precision * recall / (beta * precision + recall + K.epsilon())


        return fb
    
    loaded_model = tf.keras.models.load_model(zipped_model_path, custom_objects={"fb_score":fb_score}) 
    
    # save the model
    loaded_model.save(os.path.join(args.model_dir, "1"))
    

Overwriting train.py


In [9]:
from sagemaker.tensorflow.estimator import TensorFlow
sagemaker_session = sagemaker.Session()
boto_sess = boto3.Session()

role = get_execution_role()
default_bucket = sagemaker_session.default_bucket()
sm = boto_sess.client("sagemaker")
region = boto_sess.region_name

# arguments as Hyperparameters
bucket = "some_bucket"
prefix = "some_prefix/model.zip"
zipped_model_path = "model/model.h5"

# In this training job we're not going to use the s3_uri but we should give it anyway 
# (I haven't tried with None instead of a valid path)
inputs = {
    "train": "S3_URI_TO_DATA", 
}

estimator = TensorFlow(
    entry_point="train.py",
    model_dir=False,
    hyperparamters={
        "bucket":bucket,
        "prefix": prefix,
        "zipped_model_path": zipped_model_path
    },
    framework_version="2.6", # tf version of the original model
    py_version="py38",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    role=role,
    environment={"REGION": region}
)

#estimator.fit(inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: tensorflow-training-2023-09-08-18-10-22-960


2023-09-08 18:10:23 Starting - Starting the training job...
2023-09-08 18:10:40 Starting - Preparing the instances for training......
2023-09-08 18:11:46 Downloading - Downloading input data......
2023-09-08 18:12:47 Training - Training image download completed. Training in progress...2023-09-08 18:13:03.386509: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-09-08 18:13:03.386670: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-09-08 18:13:03.412736: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-09-08 18:13:04,538 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2023-09-08 18:13:04,546 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-09-08 18:1

In [12]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-training-2023-09-08-18-16-25-077
INFO:sagemaker:Creating endpoint-config with name tensorflow-training-2023-09-08-18-16-25-077
INFO:sagemaker:Creating endpoint with name tensorflow-training-2023-09-08-18-16-25-077


---!

## Model Registry
We're gonna register the model and for now on we should call the model from the AWS Model Registry instead of from a file.

In [27]:
import boto3
client = boto3.client("sagemaker")

In [24]:
model_package_group_name = "model-name-v1"
inference_instance_type = "ml.m4.xlarge"

model_package = estimator.register(
    model_package_group_name=model_package_group_name,
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type],
    content_types=["text/csv"],
    response_types=["text/csv"],
    approval_status="Approved",
)

model_package_arn = model_package.model_package_arn
print("Model Package ARN : ", model_package_arn)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Model Package ARN :  arn:aws:sagemaker:us-east-1:547580232856:model-package/modelo-fuga-v1/1


In [28]:
model_package_group_name = model_package_group_name
list_model_packages_response = client.list_model_packages(ModelPackageGroupName=model_package_group_name)
list_model_packages_response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'modelo-fuga-v1',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:547580232856:model-package/modelo-fuga-v1/1',
   'CreationTime': datetime.datetime(2023, 9, 8, 18, 55, 48, 252000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'}],
 'ResponseMetadata': {'RequestId': '6a46ba4f-34b6-42f9-a857-15656d739907',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6a46ba4f-34b6-42f9-a857-15656d739907',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '286',
   'date': 'Fri, 08 Sep 2023 18:58:39 GMT'},
  'RetryAttempts': 0}}

In [29]:
latest_model_version_arn = list_model_packages_response["ModelPackageSummaryList"][0][
    "ModelPackageArn"
]
print(latest_model_version_arn)

arn:aws:sagemaker:us-east-1:547580232856:model-package/modelo-fuga-v1/1
